In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
case = "fanv3_improving"
path = "/glade/scratch/jinmuluo/archive/nitrogen_cycle/lnd/hist/" 
start_date = date(2011, 1, 1)
end_date = date(2011, 12, 1)
delta = end_date - start_date

## Nitrogen Input

In [3]:
Input_vars = ['MANURE_N_APP', 'MANURE_N_GRZ', 'MANURE_N_MIX', 'FERT_N_APP', 'NDEP_TO_SMINN', 'NFIX_TO_SMINN', 
              'FFIX_TO_SMINN', 'FERT_TO_SMINN', 'NITRATE_N_TO_SMINN', 'F_CANOPY_TO_SOIL', 'area', 'landfrac']

CLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    CLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [4]:
def preprocess(ds, fields=Input_vars):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    return ds

dsCLM = fix_time(xr.open_mfdataset(CLM, decode_times=True, preprocess=preprocess))

In [5]:
MANURE_N_APP = (dsCLM["MANURE_N_APP"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
MANURE_N_GRZ = (dsCLM["MANURE_N_GRZ"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
MANURE_N_MIX = (dsCLM["MANURE_N_MIX"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
FERT_N_APP   =   (dsCLM["FERT_N_APP"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])

NDEP_TO_SMINN = (dsCLM["NDEP_TO_SMINN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NFIX_TO_SMINN = (dsCLM["NFIX_TO_SMINN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
FERT_TO_SMINN = (dsCLM["FERT_TO_SMINN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NITRATE_N_TO_SMINN = (dsCLM["NITRATE_N_TO_SMINN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
FFIX_TO_SMINN = (dsCLM["FFIX_TO_SMINN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_CANOPY_TO_SOIL = (dsCLM["F_CANOPY_TO_SOIL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])


manure_app = 0.0; manure_grz = 0.0; manure_mix = 0.0; fert_app = 0.0
ndep = 0.0; nfix = 0.0; fert_sminn = 0.0; nitrate_sminn=0.0; ffix = 0.0; canopy_soil = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(dsCLM.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    manure_app = manure_app + MANURE_N_APP[i].values * day * 24 * 3600 * 1e-12
    manure_grz = manure_grz + MANURE_N_GRZ[i].values * day * 24 * 3600 * 1e-12
    manure_mix = manure_mix + MANURE_N_MIX[i].values * day * 24 * 3600 * 1e-12
    fert_app = fert_app + FERT_N_APP[i].values * day * 24 * 3600 * 1e-12
    ndep = ndep + NDEP_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    nfix = nfix + NFIX_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    fert_sminn = fert_sminn + FERT_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    nitrate_sminn = nitrate_sminn + NITRATE_N_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    ffix = ffix + FFIX_TO_SMINN[i].values * day * 24 * 3600 * 1e-12 
    canopy_soil = canopy_soil + F_CANOPY_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    
print("manure applicaiton:", round(manure_app, 2), "  manure from grazing", round(manure_grz, 2), 
      "  manure in mix system:", round(manure_mix, 2), "  synth fertilizers:", round(fert_app, 2))
print("N input to FAN is ", manure_app + manure_grz + fert_app, "TgN/year")

print("atm depostion:", round(ndep, 2),  '  nfixation:', round(nfix, 2), "  ffxtion", int(ffix), 
      "  NH4+ diffusion", round(fert_sminn, 2), "  NO3- diffusion", round(nitrate_sminn, 2), 
    "  canopy recycle", round(canopy_soil, 2))
print("N input to CLM is ", ndep + nfix + fert_sminn + ffix + canopy_soil, "TgN/year")

manure applicaiton: 36.67   manure from grazing 65.56   manure in mix system: 92.64   synth fertilizers: 80.08
N input to FAN is  182.31566285469376 TgN/year
atm depostion: 81.98   nfixation: 114.71   ffxtion 39   NH4+ diffusion 101.05   NO3- diffusion 0.23   canopy recycle 2.36
N input to CLM is  339.9698889487406 TgN/year


## Nitrogen Ouput

In [6]:
Output_vars = ['NH3_TOTAL', 'NOx_NITRIFY_TOTAL', 'N2O_NITRIFY_TOTAL', 'NOx_DENITRIFY_TOTAL', 'N2O_DENITRIFY_TOTAL', 
              'N2_TOTAL', 'MANURE_N_TO_SMINN', 'SYNTHFERT_N_TO_SMINN', 'CANOPY_TO_SOIL', 'MANURE_NITRATE_TO_SOIL', 
              'FERT_NITRATE_TO_SOIL', 'MANURE_NH4_RUNOFF', 'FERT_NH4_RUNOFF', 'MANURE_NITRATE_RUNOFF', 'FERT_NITRATE_RUNOFF',
              'F_N2O_NIT', 'F_NOx_NIT', 'F_N2O_DENIT', 'F_NOx_DENIT', 'F_N2_DENIT', 'SMIN_NO3_RUNOFF', 
               'SMIN_NO3_LEACHED', 'area', 'landfrac']

CLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    CLM.append(path + case + ".clm2." + "h1" + "." + month +".nc")

In [7]:
def preprocess(ds, fields=Output_vars):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    return ds

dsCLM = fix_time(xr.open_mfdataset(CLM, decode_times=True, preprocess=preprocess))

In [8]:
NH3_TOTAL = (dsCLM["NH3_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NOx_NITRIFY_TOTAL = (dsCLM["NOx_NITRIFY_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
N2O_NITRIFY_TOTAL = (dsCLM["N2O_NITRIFY_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NOx_DENITRIFY_TOTAL = (dsCLM["NOx_DENITRIFY_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
N2O_DENITRIFY_TOTAL = (dsCLM["N2O_DENITRIFY_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
N2_TOTAL = (dsCLM["N2_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
MANURE_N_TO_SMINN = (dsCLM["MANURE_N_TO_SMINN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
SYNTHFERT_N_TO_SMINN = (dsCLM["SYNTHFERT_N_TO_SMINN"].fillna(0)* dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
CANOPY_TO_SOIL = (dsCLM["CANOPY_TO_SOIL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
MANURE_NITRATE_TO_SOIL = (dsCLM["MANURE_NITRATE_TO_SOIL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
FERT_NITRATE_TO_SOIL = (dsCLM["FERT_NITRATE_TO_SOIL"].fillna(0)* dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NH4_RUNOFF = ((dsCLM["MANURE_NH4_RUNOFF"].fillna(0) + dsCLM["FERT_NH4_RUNOFF"].fillna(0))* dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NITRATE_RUNOFF = ((dsCLM["MANURE_NITRATE_RUNOFF"].fillna(0) +dsCLM["FERT_NITRATE_RUNOFF"].fillna(0))* dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])

F_N2O_NIT = (dsCLM["F_N2O_NIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_NOx_NIT = (dsCLM["F_NOx_NIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_N2O_DENIT = (dsCLM["F_N2O_DENIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_NOx_DENIT = (dsCLM["F_NOx_DENIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_N2_DENIT = (dsCLM["F_N2_DENIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
SMIN_NO3_RUNOFF = (dsCLM["SMIN_NO3_RUNOFF"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
SMIN_NO3_LEACHED = (dsCLM["SMIN_NO3_LEACHED"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])

nh3 = 0.0; nox_nit = 0.0; n2o_nit = 0.0; nox_denit = 0.0; n2o_denit=0.0; n2_total = 0.0; 
manure_nh4_soil = 0.0; synthfert_nh4_soil = 0.0;
canopy_soil = 0.0; manure_no3_soil = 0.0; fert_no3_soil = 0.0 ;nh4_runoff = 0.0; no3_runoff = 0.0;
f_n2o_nit = 0.0; f_nox_nit=0.0; f_n2o_denit = 0.0; f_nox_denit = 0.0; 
f_n2_denit = 0.0; f_no3_runoff = 0.0; f_no3_leached=0.0;

# unit transfer form gN/s to Tg/year
for i in range(len(dsCLM.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    
    nh3 = nh3 + NH3_TOTAL[i].values * day * 24 * 3600 * 1e-12
    nox_nit = nox_nit + NOx_NITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    n2o_nit = n2o_nit + N2O_NITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    nox_denit = nox_denit + NOx_DENITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    n2o_denit = n2o_denit + N2O_DENITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    n2_total = n2_total + N2_TOTAL[i].values * day * 24 * 3600 * 1e-12
    manure_nh4_soil = manure_nh4_soil + MANURE_N_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    synthfert_nh4_soil = synthfert_nh4_soil + SYNTHFERT_N_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    canopy_soil = canopy_soil + CANOPY_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    manure_no3_soil = manure_no3_soil + MANURE_NITRATE_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    fert_no3_soil = fert_no3_soil + FERT_NITRATE_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    nh4_runoff = nh4_runoff + NH4_RUNOFF[i].values * day * 24 * 3600 * 1e-12
    no3_runoff = no3_runoff + NITRATE_RUNOFF[i].values * day * 24 * 3600 * 1e-12
    
    f_n2o_nit = f_n2o_nit + F_N2O_NIT[i].values * day * 24 * 3600 * 1e-12
    f_nox_nit = f_nox_nit + F_NOx_NIT[i].values * day * 24 * 3600 * 1e-12
    f_n2o_denit = f_n2o_denit + F_N2O_DENIT[i].values * day * 24 * 3600 * 1e-12
    f_nox_denit = f_nox_denit + F_NOx_DENIT[i].values * day * 24 * 3600 * 1e-12
    f_n2_denit = f_n2_denit + F_N2_DENIT[i].values * day * 24 * 3600 * 1e-12
    f_no3_runoff = f_no3_runoff + SMIN_NO3_RUNOFF[i].values * day * 24 * 3600 * 1e-12
    f_no3_leached = f_no3_leached + SMIN_NO3_LEACHED[i].values * day * 24 * 3600 * 1e-12

print("For FAN loss fluxes: Tg/yr")
print("NH3:", round(nh3, 4), "  NOx_NIT:", round(nox_nit, 4), "  NOx_DENIT:", round(nox_denit, 4),
      "  N2O_NIT:", round(n2o_nit, 4), "  N2O_DENIT:", round(n2o_denit, 4),"  N2:", round(n2_total, 4),)
print("NH4 to deep soil:", round(manure_nh4_soil + synthfert_nh4_soil, 4), 
      "  NO3 to deep soil:", round(fert_no3_soil + manure_no3_soil, 4),)
print("NH4 runoff:", round(nh4_runoff, 4), "  NO3 runoff:", round(no3_runoff, 4), "  Canopy recycle:", round(canopy_soil, 4))

print("For CLM loss fluxes: Tg/yr")
print("NOx_NIT:", round(f_nox_nit, 4), "  NOx_DENIT:", round(f_nox_denit, 4),
      "  N2O_NIT:", round(f_n2o_nit, 4), "  N2O_DENIT:", round(f_n2o_denit, 4),"  N2:", round(f_n2_denit, 4),)
print("NO3 runoff:", round(f_no3_runoff, 4), "  NO3 leached:", round(f_no3_leached, 4))

For FAN loss fluxes: Tg/yr
NH3: 47.9001   NOx_NIT: 0.6088   NOx_DENIT: 0.0007   N2O_NIT: 0.5407   N2O_DENIT: 0.0006   N2: 0.0082
NH4 to deep soil: 107.4859   NO3 to deep soil: 12.5275
NH4 runoff: 2.1058   NO3 runoff: 1.4534   Canopy recycle: 0.0688
For CLM loss fluxes: Tg/yr
NOx_NIT: 0.0826   NOx_DENIT: 6.0528   N2O_NIT: 0.062   N2O_DENIT: 4.024   N2: 41.9971
NO3 runoff: 7.6968   NO3 leached: 2.0234


In [9]:
NH3_TOTAL = (dsCLM["NH3_TOTAL"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
NOx_NITRIFY_TOTAL = (dsCLM["NOx_NITRIFY_TOTAL"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
N2O_NITRIFY_TOTAL = (dsCLM["N2O_NITRIFY_TOTAL"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
NOx_DENITRIFY_TOTAL = (dsCLM["NOx_DENITRIFY_TOTAL"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
N2O_DENITRIFY_TOTAL = (dsCLM["N2O_DENITRIFY_TOTAL"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
N2_TOTAL = (dsCLM["N2_TOTAL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
NH4_TO_SMINN = ((dsCLM["MANURE_N_TO_SMINN"].fillna(0) + dsCLM["SYNTHFERT_N_TO_SMINN"].fillna(0)) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
CANOPY_TO_SOIL = (dsCLM["CANOPY_TO_SOIL"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
NITRATE_TO_SOIL = ((dsCLM["MANURE_NITRATE_TO_SOIL"].fillna(0)+dsCLM["FERT_NITRATE_TO_SOIL"].fillna(0)) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
NH4_RUNOFF = ((dsCLM["MANURE_NH4_RUNOFF"].fillna(0) + dsCLM["FERT_NH4_RUNOFF"].fillna(0))* dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
NITRATE_RUNOFF = ((dsCLM["MANURE_NITRATE_RUNOFF"].fillna(0) +dsCLM["FERT_NITRATE_RUNOFF"].fillna(0))* dsCLM['area']*1e6).sum(dim=['lat', 'lon'])

F_N2O_NIT = (dsCLM["F_N2O_NIT"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
F_NOx_NIT = (dsCLM["F_NOx_NIT"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
F_N2O_DENIT = (dsCLM["F_N2O_DENIT"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
F_NOx_DENIT = (dsCLM["F_NOx_DENIT"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
F_N2_DENIT = (dsCLM["F_N2_DENIT"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
SMIN_NO3_RUNOFF = (dsCLM["SMIN_NO3_RUNOFF"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])
SMIN_NO3_LEACHED = (dsCLM["SMIN_NO3_LEACHED"].fillna(0) * dsCLM['area']*1e6).sum(dim=['lat', 'lon'])

nh3 = 0.0; nox_nit = 0.0; n2o_nit = 0.0; nox_denit = 0.0; n2o_denit=0.0; n2_total = 0.0; nh4_soil = 0.0;
canopy_soil = 0.0; no3_soil = 0.0; nh4_runoff = 0.0; no3_runoff = 0.0;
f_n2o_nit = 0.0; f_nox_nit=0.0; f_n2o_denit = 0.0; f_nox_denit = 0.0; 
f_n2_denit = 0.0; f_no3_runoff = 0.0; f_no3_leached=0.0;

# unit transfer form gN/s to Tg/year
for i in range(len(dsCLM.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    
    nh3 = nh3 + NH3_TOTAL[i].values * day * 24 * 3600 * 1e-12
    nox_nit = nox_nit + NOx_NITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    n2o_nit = n2o_nit + N2O_NITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    nox_denit = nox_denit + NOx_DENITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    n2o_denit = n2o_denit + N2O_DENITRIFY_TOTAL[i].values * day * 24 * 3600 * 1e-12
    n2_total = n2_total + N2_TOTAL[i].values * day * 24 * 3600 * 1e-12
    nh4_soil = nh4_soil + NH4_TO_SMINN[i].values * day * 24 * 3600 * 1e-12
    canopy_soil = canopy_soil + CANOPY_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    no3_soil = no3_soil + NITRATE_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    nh4_runoff = nh4_runoff + NH4_RUNOFF[i].values * day * 24 * 3600 * 1e-12
    no3_runoff = no3_runoff + NITRATE_RUNOFF[i].values * day * 24 * 3600 * 1e-12
    
    f_n2o_nit = f_n2o_nit + F_N2O_NIT[i].values * day * 24 * 3600 * 1e-12
    f_nox_nit = f_nox_nit + F_NOx_NIT[i].values * day * 24 * 3600 * 1e-12
    f_n2o_denit = f_n2o_denit + F_N2O_DENIT[i].values * day * 24 * 3600 * 1e-12
    f_nox_denit = f_nox_denit + F_NOx_DENIT[i].values * day * 24 * 3600 * 1e-12
    f_n2_denit = f_n2_denit + F_N2_DENIT[i].values * day * 24 * 3600 * 1e-12
    f_no3_runoff = f_no3_runoff + SMIN_NO3_RUNOFF[i].values * day * 24 * 3600 * 1e-12
    f_no3_leached = f_no3_leached + SMIN_NO3_LEACHED[i].values * day * 24 * 3600 * 1e-12

print("For FAN loss fluxes: Tg/yr")
print("NH3:", round(nh3, 4), "  NOx_NIT:", round(nox_nit, 4), "  NOx_DENIT:", round(nox_denit, 4),
      "  N2O_NIT:", round(n2o_nit, 4), "  N2O_DENIT:", round(n2o_denit, 4),"  N2:", round(n2_total, 4),)
print("NH4 to deep soil:", round(nh4_soil, 4), "  NO3 to deep soil:", round(no3_soil, 4),)
print("NH4 runoff:", round(nh4_runoff, 4), "  NO3 runoff:", round(no3_runoff, 4), "  Canopy recycle:", round(canopy_soil, 4))

print("For CLM loss fluxes: Tg/yr")
print("NOx_NIT:", round(f_nox_nit, 4), "  NOx_DENIT:", round(f_nox_denit, 4),
      "  N2O_NIT:", round(f_n2o_nit, 4), "  N2O_DENIT:", round(f_n2o_denit, 4),"  N2:", round(f_n2_denit, 4),)
print("NO3 runoff:", round(f_no3_runoff, 4), "  NO3 leached:", round(f_no3_leached, 4))

For FAN loss fluxes: Tg/yr
NH3: 69.2357   NOx_NIT: 0.882   NOx_DENIT: 0.001   N2O_NIT: 0.7356   N2O_DENIT: 0.0008   N2: 0.0082
NH4 to deep soil: 145.7933   NO3 to deep soil: 18.9966
NH4 runoff: 2.7276   NO3 runoff: 2.1477   Canopy recycle: 0.13
For CLM loss fluxes: Tg/yr
NOx_NIT: 0.1006   NOx_DENIT: 7.2777   N2O_NIT: 0.0741   N2O_DENIT: 4.8964   N2: 51.6344
NO3 runoff: 10.0166   NO3 leached: 2.3029


## Some Mid-terms

In [10]:
mt_vars = ['MANURE_NO3_TO_SOIL', 'FERT_NO3_TO_SOIL', 'F_NIT', 'F_DENIT', 'ACTUAL_IMMOB_NO3', 'ACTUAL_IMMOB_NH4', 'GROSS_NMIN', 
           'SMIN_NH4_TO_PLANT', 'SMIN_NO3_TO_PLANT', 'area', 'landfrac', 'levdcmp']

CLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    CLM.append(path + case + ".clm2." + "h2" + "." + month +".nc")

In [12]:
def preprocess(ds, fields=mt_vars):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    return ds

dsCLM = fix_time(xr.open_mfdataset(CLM, decode_times=True, preprocess=preprocess))

In [13]:
x = np.zeros(len(dsCLM['levdcmp']))
for i in range(len(x)):
    if i == 0 :
        x[i] = dsCLM['levdcmp'][i].values * 2
    else:
        x[i] = (dsCLM['levdcmp'][i].values - dsCLM['levdcmp'][i-1].values - x[i-1]/2)*2

dsCLM = dsCLM.assign(depth=(dsCLM['levdcmp'].coords, x))

MANURE_NO3_TO_SOIL = (dsCLM["MANURE_NO3_TO_SOIL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
FERT_NO3_TO_SOIL = (dsCLM["FERT_NO3_TO_SOIL"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_NIT = (dsCLM["F_NIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
F_DENIT = (dsCLM["F_DENIT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])

ACTUAL_IMMOB_NH4 = dsCLM["ACTUAL_IMMOB_NH4"].fillna(0) * dsCLM['depth']
ACTUAL_IMMOB_NH4 = (ACTUAL_IMMOB_NH4 * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['levdcmp', 'lat', 'lon'])
ACTUAL_IMMOB_NO3 = dsCLM["ACTUAL_IMMOB_NO3"].fillna(0) * dsCLM['depth']
ACTUAL_IMMOB_NO3 = (ACTUAL_IMMOB_NO3 * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['levdcmp', 'lat', 'lon'])

GROSS_NMIN = (dsCLM["GROSS_NMIN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])

SMIN_NH4_TO_PLANT = dsCLM["SMIN_NH4_TO_PLANT"].fillna(0) * dsCLM['depth']
SMIN_NH4_TO_PLANT = (SMIN_NH4_TO_PLANT  * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['levdcmp', 'lat', 'lon'])

SMIN_NO3_TO_PLANT = dsCLM["SMIN_NO3_TO_PLANT"].fillna(0) * dsCLM['depth']
SMIN_NO3_TO_PLANT = (SMIN_NO3_TO_PLANT * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['levdcmp', 'lat', 'lon'])


manure_no3_soil = 0.0; fert_no3_soil = 0.0; clm_nit = 0.0; clm_denit = 0.0; 
immob_no3 = 0.0; immob_nh4 = 0.0; mineralization=0.0; nh4_plant=0.0; no3_plant=0.0;

# unit transfer form gN/s to Tg/year
for i in range(len(dsCLM.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    
    manure_no3_soil = manure_no3_soil + MANURE_NO3_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    fert_no3_soil = fert_no3_soil + FERT_NO3_TO_SOIL[i].values * day * 24 * 3600 * 1e-12
    clm_nit = clm_nit + F_NIT[i].values * day * 24 * 3600 * 1e-12
    clm_denit = clm_denit + F_DENIT[i].values * day * 24 * 3600 * 1e-12
    immob_no3 = immob_no3 + ACTUAL_IMMOB_NO3[i].values * day * 24 * 3600 * 1e-12
    immob_nh4 = immob_nh4 + ACTUAL_IMMOB_NH4[i].values * day * 24 * 3600 * 1e-12
    mineralization = mineralization + GROSS_NMIN[i].values * day * 24 * 3600 * 1e-12
    nh4_plant = nh4_plant + SMIN_NH4_TO_PLANT[i].values * day * 24 * 3600 * 1e-12
    no3_plant = no3_plant + SMIN_NO3_TO_PLANT[i].values * day * 24 * 3600 * 1e-12

print("For FAN fluxes")
print("Nitrification:", manure_no3_soil+fert_no3_soil, "Tg/yr")
print("Denitrification:", n2o_denit+nox_denit+n2_total, "Tg/yr")

print("For CLM fluxes")
print("Nitrification:", clm_nit, "Tg/yr")
print("Denitrification:", clm_denit, "Tg/yr")
print("Immobilization from NH4+:", immob_nh4, "Tg/yr   from NO3-", immob_no3,"Tg/yr")
print("mineralization from organic pool:", mineralization, "Tg/yr")
print("plant uptake, NH4+:", nh4_plant, "Tg/yr    NO3-:", no3_plant, "Tg/yr")

For FAN fluxes
Nitrification: 38.65461428885625 Tg/yr
Denitrification: 0.01004077155022888 Tg/yr
For CLM fluxes
Nitrification: 103.38871876200001 Tg/yr
Denitrification: 54.403824090600004 Tg/yr
Immobilization from NH4+: 1778.3288637636333 Tg/yr   from NO3- 0.5345960642278597 Tg/yr
mineralization from organic pool: 2768.2844020991997 Tg/yr
plant uptake, NH4+: 1008953.8956665437 Tg/yr    NO3-: 27775791.964747034 Tg/yr


## Comparing the results from FUN

In [14]:
fun_vars = ['SMINN_TO_PLANT', 'SMINN_TO_PLANT_FUN', 'area', 'landfrac', 'levdcmp']

CLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    CLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [15]:
def preprocess(ds, fields=fun_vars):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    return ds

dsCLM = fix_time(xr.open_mfdataset(CLM, decode_times=True, preprocess=preprocess))

In [16]:
SMINN_TO_PLANT = (dsCLM["SMINN_TO_PLANT"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])
SMINN_TO_PLANT_FUN = (dsCLM["SMINN_TO_PLANT_FUN"].fillna(0) * dsCLM['area']*dsCLM['landfrac']*1e6).sum(dim=['lat', 'lon'])

normal_plant = 0.0; fun_plant = 0.0; 

# unit transfer form gN/s to Tg/year
for i in range(len(dsCLM.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    
    normal_plant = normal_plant + SMINN_TO_PLANT[i].values * day * 24 * 3600 * 1e-12
    fun_plant = fun_plant + SMINN_TO_PLANT_FUN[i].values * day * 24 * 3600 * 1e-12

print("normal scheme:", normal_plant, "Tg/yr")
print("FUN scheme:", fun_plant, "Tg/yr")

normal scheme: 19010827.368529916 Tg/yr
FUN scheme: 783.6595926335999 Tg/yr


In [17]:
nh4_plant + no3_plant - normal_plant

9773918.491883662